In [1]:
import pandas as pd
import numpy as np

In [8]:
df=pd.read_csv('train_titanic_pipeline.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [10]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [52]:
df['Embarked'].isnull().sum()

2

We can see we have the missing values in the Age column and Cabin column.We need to fill it.

# Train and test split

In [39]:
X_train,X_test,y_train,y_test=train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2, random_state=42)

In [101]:
X_train.shape

(712, 7)

In [40]:
def impute_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):

        if Pclass == 1:
            return 37

        elif Pclass == 2:
            return 29

        else:
            return 24

    else:
        return Age
        
    

In [41]:
#x_train['Age']=x_train[['Age','Pclass']].apply(impute_age,axis=1)

In [42]:
si_age = SimpleImputer()
si_embarked = SimpleImputer(strategy='most_frequent')

X_train_age = si_age.fit_transform(X_train[['Age']])
X_train_embarked = si_embarked.fit_transform(X_train[['Embarked']])

X_test_age = si_age.transform(X_test[['Age']])
X_test_embarked = si_embarked.transform(X_test[['Embarked']])

In [96]:
ohe=OneHotEncoder(sparse=False,handle_unknown='ignore')
ohe_sex = OneHotEncoder(sparse=False,handle_unknown='ignore')

In [102]:
ohe_Embarked=ohe.fit_transform(X_train[['Embarked']])
ohe_Sex=ohe_sex.fit_transform(X_train[['Sex']])
ohe_Embarked_test=ohe.transform(X_test[['Embarked']])
ohe_sex_test=ohe_sex.transform(X_test[['Sex']])

In [104]:
X_train_rem = X_train.drop(columns=['Sex','Age','Embarked'])
X_test_rem=X_test.drop(columns=['Sex','Age','Embarked'])

In [106]:
X_train_transformed=np.concatenate((X_train_rem,X_train_age,ohe_Sex,ohe_Embarked),axis=1)
X_test_transformed=np.concatenate((X_test_rem,X_test_age,ohe_sex_test,ohe_Embarked_test),axis=1)

In [108]:
X_train_transformed.shape

(712, 11)

In [110]:
clf = DecisionTreeClassifier()
clf.fit(X_train_transformed,y_train)

DecisionTreeClassifier()

In [112]:
y_pred=clf.predict(X_test_transformed)

In [113]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7821229050279329

In [114]:
import pickle

In [122]:
pickle.dump(ohe,open('MODELS/ohe.pkl','wb'))

In [119]:
pickle.dump(ohe_sex,open('MODELS/ohe_sex.pkl','wb'))

In [121]:
pickle.dump(clf,open('MODELS/clf.pkl','wb'))